##### Copyright 2020 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Настройка того, что происходит в `fit()`

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/keras-team/keras-io/blob/master/tf/customizing_what_happens_in_fit.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/keras-team/keras-io/blob/master/guides/customizing_what_happens_in_fit.py"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/keras-io/tf/customizing_what_happens_in_fit.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Введение

Когда вы занимаетесь обучением под наблюдением, вы можете использовать `fit ()`, и 
все работает гладко.

Когда вам нужно написать собственный цикл обучения с нуля, вы можете использовать
'GradientTape` и взять под контроль каждую мелочь.

Но что, если вам нужен собственный алгоритм обучения, а вы все равно хотите извлечь 
выгоду из удобной для Вас функции `fit ()`, а также из обратные вызовы (callbacks), 
встроенная поддержка распространения (built-in distribution support), 
или пошаговое слияние (step fusing)?

Основным принципом Keras является ** прогрессивное раскрытие сложности **. Вам следует
всегда иметь возможность постепенно переходить к рабочим процессам более низкого уровня. 
Не надо отворачиваться от возможностей Keras, если функциональность высокого уровня не 
совсем соответствует вашему варианту использования. Надо достигать состояния большего 
контроля над мелкими ньюансами и иметь общую выгоду от работы на высоком уровне.

Чтобы настроить `fit ()`, вы должны ** переопределить функцию шага обучения в 
классе `Model` **. Это функция, которая вызывается методом `fit ()` для каждого пакета 
данных. После переопределения вы сможете вызывать `fit ()` как обычно - и он будет 
запускать ваш собственный алгоритм обучения.

Обратите внимание, что этот шаблон не мешает вам строить модели с функциональными
API. Вы можете делать это независимо от того, строите ли вы модели класса `Sequential`, модели Functional API модели типа subclass.

Теперь посмотрим как это будет работать.

## Setup
Требуется  TensorFlow 2.2 or later.

In [0]:
import tensorflow as tf
from tensorflow import keras

## Первый простой пример

Давайте наченм с простого примера:

- Мы создадим новый класс от  класса `keras.Model`.
- Переопределяем метод `train_step(self, data)`.
- Возвращаем словарь, отображающий имена метрик (включая потери) dictionary mapping 
metric names (including the loss)) в их текущее значение.

The input argument `data` is what gets passed to fit as training data:

- IЕсли на вход подаемNumpy arrays, bвызывая функцию`fit(x, y, ...)`, tтогда`data` w
должна быть кортежем (x, y)`
- IЕсли на вход подаем`tf.data.Dataset`, bвызывая функцию`fit(dataset, ...)`, tтогда`
data` wбудет что называется`dataset` aна каждой партии atch.

IВнутри метода `train_step` реализуется регулярное обновление обучения, похожее на то, 
с чем вы уже знакомы. Важно отметить, что ** вычисляется потеря с помощью `self.compiled_loss` **, это обертка для функцию потерь, которая была передана в ` compile () `

Точно так же мы вызываем `self.compiled_metrics.update_state (y, y_pred)`, чтобы обновить состояние метрик, которые были переданы в `compile ()`, и запрашиваем результаты из `self.metrics` в конце, чтобы получить их текущие значение

In [0]:
class CustomModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}


Теперь попробуем это:

In [0]:
import numpy as np

# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Just use `fit` as usual
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=3)

## Переходим на низкий уровень

Естественно, можно просто пропустить передачу функции потерь в `compile ()` и вместо этого делать все * вручную * в `train_step`. Аналогично это и для метрик. Вот пример более низкого уровня, который использует только `compile ()` для настройки оптимизатора:

In [0]:
mae_metric = keras.metrics.MeanAbsoluteError(name="mae")
loss_tracker = keras.metrics.Mean(name="loss")


class CustomModel(keras.Model):
    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute our own loss
            loss = keras.losses.mean_squared_error(y, y_pred)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Compute our own metrics
        loss_tracker.update_state(loss)
        mae_metric.update_state(y, y_pred)
        return {"loss": loss_tracker.result(), "mae": mae_metric.result()}


# Construct an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)

# We don't passs a loss or metrics here.
model.compile(optimizer="adam")

# Just use `fit` as usual -- you can use callbacks, etc.
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=3)

## Поддержка `sample_weight` & `class_weight`

Возможно, вы заметили, что в нашем первом базовом примере не упоминалось взвешивание 
выборки. Если вы хотите поддерживать такие аргументы `fit()` как  `sample_weight` и
`class_weight`, надо сделать следующее:

- Распаковать `sample_weight` из аргумента `data` 
- Передать его в `compiled_loss` & `compiled_metrics` (можно применить его и вручную если
не полагаться на  `compile()` для потерь и метрик)
- Все, вот этот список.

In [0]:
class CustomModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        if len(data) == 3:
            x, y, sample_weight = data
        else:
            x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value.
            # The loss function is configured in `compile()`.
            loss = self.compiled_loss(
                y,
                y_pred,
                sample_weight=sample_weight,
                regularization_losses=self.losses,
            )

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics.
        # Metrics are configured in `compile()`.
        self.compiled_metrics.update_state(y, y_pred, sample_weight=sample_weight)

        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# You can now use sample_weight argument
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
sw = np.random.random((1000, 1))
model.fit(x, y, sample_weight=sw, epochs=3)

## Предоставление собственного шага оценки

Надо сделать тоже самое для `model.evaluate()`? Тогда надо таким же способом 
переопределить `test_step` . Вот как это сделать:

In [0]:
class CustomModel(keras.Model):
    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False)
        # Updates the metrics tracking the loss
        self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


# Construct an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(loss="mse", metrics=["mae"])

# Evaluate with our custom test_step
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.evaluate(x, y)

## Подведение итогов: пример GAN от начала до конца

Давайте рассмотрим сквозной пример, который использует все, что вы только что узнали.

Давайте рассмотрим:

- Сеть генератора для генерации изображений размером 28x28x1.
- Сеть дискриминаторов, предназначенная для классификации изображений 28x28x1 на два класса («фейковые» и "реальные").
- Один оптимизатор для каждого.
- Функция потери для обучения дискриминатора.

- A generator network meant to generate 28x28x1 images.
- A discriminator network meant to classify 28x28x1 images into two classes ("fake" and
"real").
- One optimizer for each.
- A loss function to train the discriminator.


In [0]:
from tensorflow.keras import layers

# Create the discriminator
discriminator = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)

# Create the generator
latent_dim = 128
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        # We want to generate 128 coefficients to reshape into a 7x7x128 map
        layers.Dense(7 * 7 * 128),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, 128)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

Вот полнофункциональный класс GAN, переопределяющий `compile ()` для использования собственной сигнатуры, и реализующий весь алгоритм GAN в 17 строках в `train_step`:

In [0]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_images):
        if isinstance(real_images, tuple):
            real_images = real_images[0]
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))
        return {"d_loss": d_loss, "g_loss": g_loss}


Теперь запустим это:

In [0]:
# Prepare the dataset. We use both the training & test MNIST digits.
# Приготовление датасета. Используются оба -  training & test MNIST значения.
batch_size = 64
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_digits = all_digits.astype("float32") / 255.0
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))
dataset = tf.data.Dataset.from_tensor_slices(all_digits)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

# To limit execution time, we only train on 100 batches. You can train on
# the entire dataset. You will need about 20 epochs to get nice results.
# Для ограничения времени выполнения берутся только 100 партий. Можно тренировать
# на всем датасете. Надо будет выставить 20 эпох для хорошего результата.
gan.fit(dataset.take(100), epochs=1)

Идея глубокого обучения проста, так почему их реализация должна быть болезненной?